# 梯度检验
梯度检验用来调试神经网络的反向传播部分  

**其原理为：**
- 在很小的区间内，数值梯度近似等于解析梯度
- 前向传播比较简单，因此基本能保证前向传播不出现错误，也就保证最后的计算结果不出错，而数值梯度的计算依托于最后的计算结果，也就基本保证数值梯度的计算不出错   

数值梯度：$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon}$

一般在实现反向传播时我们使用解析梯度，这时通过对比其与数值梯度的差别来检验反向传播过程中是否有计算错误

In [3]:
import numpy as np
from depends.testCases import *
from depends.gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

## 一维梯度检验
对于一维线性函数 $J(\theta) = \theta x$ 进行梯度检验  
<img src = "source\1-D gradient checking.png"></img>

In [4]:
def forward_propagation(x, theta):
    
    J = theta * x
    
    return J

In [5]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

J = 8


求反向传播部分，对于线性函数，$\frac{\partial J}{\partial \theta} = x$

In [8]:
def backward_propagation(x, theta):

    dtheta = x
    
    return dtheta

In [9]:
x, theta = 2, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

dtheta = 2


## 数值梯度的定义
$\theta^{+} = \theta + \varepsilon$  
$\theta^{-} = \theta - \varepsilon$  
$J^{+} = J(\theta^{+})$  
$J^{-} = J(\theta^{-})$  
$gradapprox = \frac{J^{+} - J^{-}}{2  \varepsilon}$  

$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2}$  

分子可以用`np.linalg.norm(...)`计算  
如果算出的difference很小(小于10^-7)，说明反向传播计算正确

In [12]:
def gradient_check(x, theta, epsilon = 1e-7):

    thetaplus = theta + epsilon                   
    thetaminus = theta - epsilon                             
    J_plus = forward_propagation(x, thetaplus)
    J_minus = forward_propagation(x, thetaminus)                                 
    gradapprox = (J_plus - J_minus) / (2 * epsilon)                             

    grad = backward_propagation(x, theta)

    numerator = np.linalg.norm(grad - gradapprox)                              
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)                     
    difference = numerator / denominator                        

    if difference < 1e-7:
        print ("The gradient is correct!")
    else:
        print ("The gradient is wrong!")
    
    return difference

In [13]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

The gradient is correct!
difference = 2.91933588329e-10


## N维梯度检验
<img src = "source\N-D gradient checking.png"></img>

In [14]:
def forward_propagation_n(X, Y, parameters):

    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    logprobs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1./m * np.sum(logprobs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

In [15]:
def backward_propagation_n(X, Y, cache):

    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T) * 2
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 4./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

这里的 $\theta$ 不是一个标量，而是一个字典类型，包含了所有的W与b  
本实验提供了一个函数`dictionary_to_vector()`，将其转换为一个向量，`vector_to_dictionary()`将其转换回字典  
<img src = "source\dictionary to vector.png"></img>  

- 首先令$\theta^{+}$ = `np.copy(parameters_values)`
- 然后计算$\theta^{+}_i$ = $\theta^{+}_i + \varepsilon$
- 最后计算$J^{+}_i$ = `forward_propagation_n(x, y, vector_to_dictionary(`$\theta^{+}$ `))`.

In [77]:
def gradient_check_n(parameters, gradients, X, Y, epsilon = 1e-7):

    parameters_values, _ = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))

    for i in range(num_parameters):

        thetaplus = np.copy(parameters_values)                                
        thetaplus[i] = thetaplus[i] + epsilon                           
        J_plus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaplus))                                  

        thetaminus = np.copy(parameters_values)                              
        thetaminus[i] = thetaminus[i] - epsilon                             
        J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus))                             

        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)

    numerator = np.linalg.norm(grad - gradapprox)                              
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)                     
    difference = numerator / denominator                                       

    if difference > 2e-7:
        print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
        print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
    return difference

In [78]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y)

There is a mistake in the backward propagation! difference = 0.285093156654
